In [1]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4

  Preparing metadata (setup.py) ... done
  Created wheel for colored: filename=colored-1.4.4-py3-none-any.whl size=14248 sha256=110676b80bfb35beaf9e60e8d73481148b4a3b21587c63574d78ccfa2773a2a0
  Stored in directory: /root/.cache/pip/wheels/e7/c3/07/fabb0941ff5df7a487d43a67081273045536cc96d4d0e816b4
Successfully built colored
  Attempting uninstall: colored
    Found existing installation: colored 2.2.3
    Uninstalling colored-2.2.3:
      Successfully uninstalled colored-2.2.3


In [2]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

Rozwiązanie zagadnienia kinematyki odwrotnej, podobnie jak zagadnienia kinematyki prostej, trzeba rozpocząć od stworzenia modelu manipulatora.

In [3]:
L1 = rtb.DHLink(d=0.645, alpha=pi/2, theta=0.0, a=0.330,)
L2 = rtb.DHLink(d=0.0, alpha=0.0, theta=0.0, a=1.35)
L3 = rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.115)
L4= rtb.DHLink(d=1.420, alpha=-pi/2, theta=0.0, a=0.0)
L5= rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.0)
L6= rtb.DHLink(d=0.215, alpha=0.0, theta=0.0, a=0.0)
robot = rtb.DHRobot([L1, L2, L3, L4, L5, L6])

W następnym kroku konieczne jest określenie położenia i orientacji końcówki manipulatora, dla której zagadnienie  ma zostać rozwiązane. Pozycję tą należny przedstawić w postaci macierzy homogenicznej. Jednym z najprostszych sposobów jest określenie macierzy translacji i rotacji. <br>
Macierz translacji stworzymy obliczając ją dla przykładu z kinematyki prostej. W ten sposób manipulator ustawi się ostatecznie w tym samym miejscu.

Rozwiązanie rozwiązania zagadnienia kinematyki odwrotnej można otrzymać wywołując na obiekcie robota metodę **ikine_LM** i podając jako argument macierz jednorodną.
```py
sol = robot.ikine_LM(T)
```


In [4]:
trans = SE3(-1, 1, 2)
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
T = trans * rot
mask = np.array([1, 1, 1, 0, 0, 0])
sol = robot.ikine_LM(T, mask=mask)
sol

IKSolution(q=array([ -0.6952,    3.122,   0.1032,  -0.6685,     1.87,   -1.046]), success=True, iterations=6, searches=1, residual=8.184912621448038e-11, reason='Success')

In [8]:
trans_test = SE3(0.5, -0.5, -1)
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
T = trans_test * rot
mask = np.array([1, 1, 1, 0, 0, 0])
sol = robot.ikine_LM(T, mask=mask)
print(sol)

IKSolution: q=[-0.7786, -2.135, -3.088, 2.866, 3.059, -0.9554], success=True, iterations=6, searches=1, residual=1.3e-12


In [9]:
my_trans_co = [
               (0.5, 0.5, -1),
               (0.5, 0.3, -1),
               (0.5, 0.1, -1),
               (0.5, -0.1, -1),
               (0.5, -0.3, -1),
               (0.5, -0.5, -1)]


In [16]:
def calculate_sol(x):
  trans = SE3(x)
  y = [0,0,1]
  z = [1,0,0]
  rot = SE3.OA(y, z)
  T = trans * rot
  mask = np.array([1, 1, 1, 0, 0, 0])
  sol = robot.ikine_LM(T, mask=mask)
  print(sol.success)
  if sol.success == True:
    return sol.q


In [17]:
qr = []
for i in range(len(my_trans_co)):
  my_sol = calculate_sol(my_trans_co[i])
  qr.append(my_sol)
q_array = np.array(qr)
print(q_array)

True
True
True
True
True
True
[[  0.8367   -2.146   -3.129     0.69    2.873   -2.216]
 [  -2.908   -1.128  -0.1919    1.932    1.066    2.919]
 [-0.08801   -2.569   -2.844    1.715  -0.7407    1.362]
 [-0.01723    -2.48   -3.024   -2.698   -1.433  -0.9418]
 [ -0.5498  -0.2853  -0.4765    2.194 -0.03117   -1.543]
 [   2.597   -1.217  -0.1323   -1.976    1.023     2.79]]


In [18]:
robot.plot(q_array, movie="robot_ikine.gif")

PyPlot3D backend, t = 0.3, scene:
  robot: Text(0.0, 0.0, '')

In [19]:
with open("wynik.txt", "w") as plik:
  for q in q_array:
    print(q)
    plik.write(','.join(map(str,q)))
    plik.write('\n')

[  0.8367   -2.146   -3.129     0.69    2.873   -2.216]
[  -2.908   -1.128  -0.1919    1.932    1.066    2.919]
[-0.08801   -2.569   -2.844    1.715  -0.7407    1.362]
[-0.01723    -2.48   -3.024   -2.698   -1.433  -0.9418]
[ -0.5498  -0.2853  -0.4765    2.194 -0.03117   -1.543]
[   2.597   -1.217  -0.1323   -1.976    1.023     2.79]


In [20]:
import pandas as pd
read_file = pd.read_csv ('wynik.txt')
read_file.to_csv ('wynik_w_csv.csv', index=None)
df = pd.read_csv('wynik_w_csv.csv')
# dodajemy naglowki
df.columns =['q1','q2','q3','q4','q5','q6']
# jaka dlugosc csv
rows_in_csv = df.shape[0]
# do takiej wartosci for tworzy liste ze skokiem 0.1
tt = []
for x in range(0, rows_in_csv):
  t = x*0.1
  tt.append(t)
# dodajemy to jako pierwsza kolumna
df['time'] = tt
df = df[['time', 'q1','q2','q3','q4','q5','q6']]
df.to_csv('updated_wynik_w_csv.csv', index=False)